<a href="https://colab.research.google.com/github/marcio-mutti/data_science_iesb/blob/main/Disc_Soft_Trabalho_Encerramento_Semestre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install odfpy
!pip install folium --upgrade

     |████████████████████████████████| 727kB 3.1MB/s 
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160691 sha256=a0124f20267f0be63cb3c843f7253bcebb93445bf1a9a9f7d362437066c23e15
  Stored in directory: /root/.cache/pip/wheels/ce/6b/93/ec330f2991c5f1546abf640360e5a2022f76cb16e5d99ed2fd
Successfully built odfpy
     |████████████████████████████████| 102kB 2.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.12.1 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sbn
import folium as fl
from matplotlib import pyplot as plt
from matplotlib.axes import Axes
from scipy import stats
from sklearn import linear_model
from requests import Session
from zipfile import ZipFile
from datetime import date, datetime, timedelta
from io import BytesIO
from math import ceil
from bz2 import decompress
from json import loads as JSONloads, dumps as JSONdumps
from shapely.geometry import asShape

In [ ]:
def colorizar_county(f):
  cor='#ffffff'
  resultado = {'fillOpacity': 0.6, 'weight': 0}
  td = f['properties']['tend']
  if td != '-':
    cor = '#ffa500'
    if td < e_c:
      cor = '#d62728'
    elif td < e_m:
      cor = '#2e8b57'
    elif td < e_a:
      cor = '#778899'
  resultado['fillColor']=cor
  return resultado

def calcular_derivada(z):
  regressor = linear_model.LinearRegression()
  datas = np.array([datetime.strptime(x + '-01', '%Y-%m-%d').date() for x in z['year_month']])
  redatas = np.array([(x - datas.min()).days for x in datas]).reshape(-1,1)
  regressor.fit(X=redatas, y=z['excedente'])
  return regressor.coef_[0]

def situar_cidades(vendas_realizadas: pd.DataFrame, estoque_util: pd.DataFrame) -> pd.DataFrame:
  expansao_vendas = pd.merge(vendas_realizadas, lojas, on=['storeid'], how='left')
  vendas_cidades = expansao_vendas.groupby(by=['year_month', 'city', 'state']).agg({'sales_qty': 'sum'}).reset_index()
  movimentacao_estoque = pd.merge(vendas_cidades, estoque_util, on=['city', 'state', 'year_month'], how='outer')
  movimentacao_estoque.loc[:, 'supplied_qty'] = movimentacao_estoque.loc[:, 'supplied_qty'].fillna(0).astype(np.int32)
  cidades_cujos_estoques_nao_diminuiram=movimentacao_estoque[movimentacao_estoque['sales_qty'].isna()]['city']
  for cid in cidades_cujos_estoques_nao_diminuiram:
    if len(cidades.loc[cid]) > 1:
      print(f"A cidade {cid} possui homônimas em outros estados")
    elif len(cidades.loc[cid])==0:
      raise RuntimeError('Tem de ver isso aqui.')
    else:
      movimentacao_estoque.loc[(movimentacao_estoque['city']==cid)&(movimentacao_estoque['state'].isna()),'state']=cidades.loc[cid].item()
      movimentacao_estoque.loc[(movimentacao_estoque['city'] == cid) & (movimentacao_estoque['sales_qty'].isna()), 'sales_qty'] = 0
  movimentacao_estoque.loc[:, 'sales_qty'] = movimentacao_estoque.loc[:, 'sales_qty'].fillna(0).astype(np.int32)
  movimentacao_estoque['venda_acumulada'] = movimentacao_estoque.groupby(by=['city', 'state'])['sales_qty'].transform(pd.Series.cumsum)
  movimentacao_estoque['estoque_acumulado'] = movimentacao_estoque.groupby(by=['city','state'])['supplied_qty'].transform(pd.Series.cumsum)
  movimentacao_estoque['excedente'] = movimentacao_estoque['estoque_acumulado'] - movimentacao_estoque['venda_acumulada']
  return pd.merge(movimentacao_estoque.groupby(by=['city', 'state']).apply(calcular_derivada).rename('tendencia').reset_index(), zipes, on=['city', 'state'], how='left')

def plotar_histograma_da_situacao_do_estoque_das_cidades(situacao_cidades: pd.DataFrame, axis: Axes):
  e_c = 0
  e_m = 0.1
  e_a = 0.5
  sbn.histplot(data=situacao_cidades, x='tendencia', binwidth=0.05, ax=axis)
  axis.set_ylabel("N° de cidades")
  for pat in axis.patches:
    if pat.get_x() < e_c:
      pat.set_ec('darkred')
      pat.set_fc('tab:red')
    elif pat.get_x() < e_m:
      pat.set_ec('darkgreen')
      pat.set_fc('seagreen')
    elif pat.get_x() < e_a:
      pat.set_ec('darkslategrey')
      pat.set_fc('lightslategrey')
    else:
      pat.set_ec('darkorange')
      pat.set_fc('orange')

def plotar_vendas_diarias_e_estoque(quadro_geral_de_vendas: pd.DataFrame, quadro_geral_de_estoque: pd.DataFrame, axes: list):
  vendas_resumo = quadro_geral_de_vendas.groupby(by=['date_of_sales', 'product']).agg(vendas=('sales_qty', 'sum')).reset_index()
  estoque_resumo = pd.merge(quadro_geral_de_estoque.groupby(by='year_month').agg({'supplied_qty': 'sum'}), quadro_geral_de_vendas.groupby(by=['year_month']).agg({'sales_qty': 'sum'}), left_index=True, right_index=True, how='inner').reset_index()
  estoque_resumo['data_ref'] = estoque_resumo.loc[:, 'year_month'].apply(lambda z: pd.to_datetime(z + '-15'))
  estoque_resumo['saldo'] = estoque_resumo['supplied_qty'] - estoque_resumo['sales_qty']
  sbn.lineplot(data=vendas_resumo, x='date_of_sales', y='vendas', hue='product', palette='Set2', ax=axes[0])
  sbn.lineplot(data=estoque_resumo, x='data_ref', y='supplied_qty', ax=axes[1], color='tab:cyan')
  axt = axes[1].twinx()
  sbn.lineplot(data=estoque_resumo, x='data_ref', y='saldo', ax=axt, color='tab:red')

In [ ]:
#Carregamento de dados
with Session() as sessao:
  download=sessao.get('https://github.com/marcio-mutti/data_science_iesb/raw/main/Python_1/trabalho_final_software_001.zip')
  if download.status_code != 200:
    raise RuntimeError("Não foi possível recuperar arquivos do trabalho.")
  download_mapa_counties = sessao.get('https://github.com/marcio-mutti/data_science_iesb/raw/main/Mapas/county.geojson.zip')
  if download_mapa_counties.status_code != 200:
    raise RuntimeError("Não foi possível recuperar mapa de zipcodes dos EUA")
  with ZipFile(BytesIO(download.content)) as pacote:
    with pacote.open('sales_data.csv') as arquivo_vendas:
      vendas=pd.read_csv(arquivo_vendas,sep='\t', names=['storeid','date_of_sales','product','sales_qty'],parse_dates=[1]).sort_values(by=['date_of_sales','storeid','product'])
    with pacote.open('store_data.csv') as arquivo_lojas:
      lojas=pd.read_csv(arquivo_lojas,sep='\t',names=['storeid','city','state','zip'])
    with pacote.open('supply_data.csv') as arquivo_estoque:
      estoque = pd.read_csv(arquivo_estoque, sep='\t', names=['year_month', 'city', 'supplied_qty'])        
  print('Dados da empresa recuperados.')
  with ZipFile(BytesIO(download_mapa_counties.content)) as pacote:
    with pacote.open('county.geojson') as empacotado:
      mapa_counties=JSONloads(empacotado.read())
  print('Mapa de condados recuperado')
zip_counties = pd.read_excel('https://github.com/marcio-mutti/data_science_iesb/raw/main/Mapas/uszipcounties.ods', engine='odf')
zip_counties.loc[:, 'county_fips'] = zip_counties.loc[:, 'county_fips'].apply(lambda z: f"{z:05n}").apply(lambda z: z[:2] + '-' + z[2:])
zipes = pd.merge(lojas.drop(labels=['storeid'], axis=1),zip_counties.drop(labels=['city','state_id'],axis=1),on=['zip'],how='left')
print('Correlação de zips e condados recuperada.')
populacao=pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/cities/totals/sub-est2019_all.csv',usecols=[8,9,21], encoding='ISO-8859-1')
print("Dados de população recuperados.")
ztca=pd.read_csv('https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_cousub_rel_10.txt')
print("Dados de house income recuperados.")
vendas['year_month']=vendas['date_of_sales'].dt.strftime('%Y-%m')
populacao['city']=populacao['NAME'].str.upper()
print('Todos os dados recuperados.')

Dados da empresa recuperados.
Mapa de condados recuperado
Correlação de zips e condados recuperada.
Dados de população recuperados.
Dados de house income recuperados.
Todos os dados recuperados.


In [ ]:
#Missing data??
print("Dados ausentes.")
print("Vendas: {}, Lojas: {}, Estoque: {}.".format(vendas.isna().sum().sum(),lojas.isna().sum().sum(), estoque.isna().sum().sum()))

Dados ausentes.
Vendas: 0, Lojas: 0, Estoque: 0.


In [ ]:
#cidades e estados
cidades=lojas[['city','state']].drop_duplicates().set_index('city')
cidades.head(5)

,state
city,
FARMINGTON,MO
NAUGATUCK,CT
FOREST LAKE,MN
OCALA,FL
LUDINGTON,MI


In [ ]:
#Remover informações de vendas zeradas
vendas_zeradas=vendas[vendas['sales_qty']==0].copy()
vendas.drop(vendas[vendas['sales_qty']==0].index,inplace=True)
vendas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193999 entries, 860746 to 796255
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   storeid        193999 non-null  int64         
 1   date_of_sales  193999 non-null  datetime64[ns]
 2   product        193999 non-null  object        
 3   sales_qty      193999 non-null  int64         
 4   year_month     193999 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 8.9+ MB


In [ ]:
#Lojas em municípios homônimos
def contar_cidades(z):
  return len(lojas_mun_homonimos[lojas_mun_homonimos['city']==z['city']])
lojas_mun_homonimos=lojas[lojas.duplicated(subset=['city'],keep=False)].sort_values(by=['city','state'])
lojas_mun_homonimos.drop_duplicates(subset=['city','state'], inplace=True)
lojas_mun_homonimos['n_entradas']=lojas_mun_homonimos.apply(contar_cidades,axis=1)
lojas_mun_homonimos.drop(lojas_mun_homonimos[lojas_mun_homonimos['n_entradas']==1].index,inplace=True)
lojas_mun_homonimos.drop(labels=['n_entradas'], axis=1,inplace=True)
lojas_mun_homonimos

,storeid,city,state,zip
3203,1968,ABERDEEN,MD,21001
3709,1097,ABERDEEN,NC,28315
4293,1520,ABERDEEN,SD,57401
763,2037,ABERDEEN,WA,98520
270,5797,ALBANY,GA,31705
...,...,...,...,...
1805,3434,WOODSTOCK,IL,60098
2161,2647,WOODSTOCK,VA,22664
2138,350,YORK,NE,68467
1649,1529,YORK,PA,17402


In [ ]:
#Vendas mensais municipais
vendas_lojas=pd.merge(vendas,lojas,on='storeid',how='inner')
vendas_mensais_cidades = vendas_lojas.groupby(by=['city','state','year_month']).agg({'sales_qty':'sum'}).reset_index()
vendas_mensais_cidades['perc']=0
cidades_problematicas=lojas_mun_homonimos['city'].unique()
for cid in vendas_mensais_cidades['city'].unique():
  for ym in vendas_mensais_cidades[vendas_mensais_cidades['city']==cid]['year_month'].unique():
    total_de_vendas=vendas_mensais_cidades[(vendas_mensais_cidades['city']==cid)&(vendas_mensais_cidades['year_month']==ym)]['sales_qty'].sum()
    vendas_mensais_cidades.loc[(vendas_mensais_cidades['city']==cid)&(vendas_mensais_cidades['year_month']==ym),'perc']=vendas_mensais_cidades[(vendas_mensais_cidades['city']==cid)&(vendas_mensais_cidades['year_month']==ym)]['sales_qty']/total_de_vendas
vendas_mensais_cidades

,city,state,year_month,sales_qty,perc
0,ABBEVILLE,LA,2019-05,17,1.000000
1,ABBEVILLE,LA,2019-06,6,1.000000
2,ABBEVILLE,LA,2019-07,11,1.000000
3,ABERDEEN,MD,2019-05,15,0.145631
4,ABERDEEN,MD,2019-06,8,0.085106
...,...,...,...,...,...
9135,ZEPHYRHILLS,FL,2019-06,10,1.000000
9136,ZEPHYRHILLS,FL,2019-07,23,1.000000
9137,ZION,IL,2019-05,14,1.000000
9138,ZION,IL,2019-06,9,1.000000


In [ ]:
#Agora, um estoque estimado
novo_estoque = pd.DataFrame()
for ym in estoque['year_month'].unique():
  for cid in estoque.loc[estoque['year_month']==ym,'city']:
    reestoque=estoque.loc[(estoque['year_month']==ym)&(estoque['city']==cid)]['supplied_qty']
    reestoque = reestoque.item() if len(reestoque>0) else 0
    deestoque=0
    vendas_local = vendas_mensais_cidades.loc[(vendas_mensais_cidades['year_month']==ym)&(vendas_mensais_cidades['city']==cid)].sort_values(by=['sales_qty'],ascending=[True])
    saldo=0
    n_iter=len(vendas_local)
    reg=None
    n_cid=0
    if n_iter == 0: #Não houve venda nessa cidade nesse mês
      n_cid=len(cidades.loc[cid])
      if n_cid == 1:
        novo_estoque=pd.concat([novo_estoque,pd.DataFrame({'year_month':ym,'city':cid, 'state':cidades.loc[cid], 'supplied_qty':reestoque},index=[0])],ignore_index=True)
      else:
        for i, st in enumerate([x[0] for x in cidades.loc[cid].to_numpy()]):
          if i < n_cid - 1:
            deestoque=ceil(reestoque/n_cid)
          else:
            deestoque=reestoque-saldo
          saldo += deestoque
          novo_estoque=pd.concat([novo_estoque,pd.DataFrame({'year_month':ym,'city':cid, 'state':st, 'supplied_qty':reestoque},index=[0])],ignore_index=True)
    else:
      for i,reg in enumerate([x[1] for x in vendas_local.iterrows()]):
        if i < n_iter-1:
          deestoque = ceil(reg['perc']*reestoque)
        else:
          deestoque=reestoque-saldo # Para a cidade com a maior venda, um ajuste de eventuais arredondamentos, pois a variável é discreta
        saldo+=deestoque
        novo_estoque=pd.concat([novo_estoque,pd.DataFrame({'year_month':ym,'city':cid, 'state':reg['state'], 'supplied_qty':deestoque}, index=[0])],ignore_index=True)
      assert(saldo-reestoque==0)
print(f"Total estoque original: {estoque['supplied_qty'].sum()}. Total estoque estimado: {novo_estoque['supplied_qty'].sum()}. Diferença de {abs(estoque['supplied_qty'].sum()-novo_estoque['supplied_qty'].sum())}")
assert(estoque['supplied_qty'].sum()==novo_estoque['supplied_qty'].sum())

Total estoque original: 592588. Total estoque estimado: 592588. Diferença de 0


In [ ]:
teste=pd.merge(novo_estoque.groupby(by=['year_month','city']).agg({'supplied_qty':'sum'}).reset_index(),estoque,on=['year_month','city'],how='outer',suffixes=('_novo','original'))
teste[teste['supplied_qty_novo']!=teste['supplied_qtyoriginal']] #Conferir se alguma cidade ficou com reestocagem incorreta comparando com sua informação original

,year_month,city,supplied_qty_novo,supplied_qtyoriginal


In [ ]:
#Movimentação do estoque
def calcular_derivada(z):
  regressor = linear_model.LinearRegression()
  datas = np.array([datetime.strptime(x + '-01', '%Y-%m-%d').date() for x in z['year_month']])
  redatas = np.array([(x - datas.min()).days for x in datas]).reshape(-1,1)
  regressor.fit(X=redatas, y=z['excedente'])
  return regressor.coef_[0]
  
expansao_vendas=pd.merge(vendas,lojas,on=['storeid'],how='left')
vendas_cidades=expansao_vendas.groupby(by=['year_month','city','state']).agg({'sales_qty':'sum'}).reset_index()
movimentacao_estoque = pd.merge(vendas_cidades,novo_estoque, on=['city','state','year_month'],how='outer')
movimentacao_estoque.loc[:,'supplied_qty']=movimentacao_estoque.loc[:,'supplied_qty'].fillna(0).astype(np.int32) #Onde não há registro de reestoque, vale a pena considerar reestoque igual a 0
cidades_cujos_estoques_nao_diminuiram=movimentacao_estoque[movimentacao_estoque['sales_qty'].isna()]['city']
for cid in cidades_cujos_estoques_nao_diminuiram:
  if len(cidades.loc[cid]) > 1:
    print(f"A cidade {cid} possui homônimas em outros estados")
  elif len(cidades.loc[cid])==0:
    raise RuntimeError('Tem de ver isso aqui.')
  else:
    movimentacao_estoque.loc[(movimentacao_estoque['city']==cid)&(movimentacao_estoque['state'].isna()),'state']=cidades.loc[cid].item()
    movimentacao_estoque.loc[(movimentacao_estoque['city']==cid)&(movimentacao_estoque['sales_qty'].isna()),'sales_qty']=0
movimentacao_estoque.loc[:, 'sales_qty'] = movimentacao_estoque.loc[:, 'sales_qty'].astype(np.int32)
movimentacao_estoque['venda_acumulada'] = movimentacao_estoque.groupby(by=['city', 'state'])['sales_qty'].transform(pd.Series.cumsum)
movimentacao_estoque['estoque_acumulado'] = movimentacao_estoque.groupby(by=['city','state'])['supplied_qty'].transform(pd.Series.cumsum)
movimentacao_estoque['excedente'] = movimentacao_estoque['estoque_acumulado'] - movimentacao_estoque['venda_acumulada']
situacao_cidades = pd.merge(movimentacao_estoque.groupby(by=['city','state']).apply(calcular_derivada).rename('tendencia').reset_index(),zipes,on=['city','state'],how='left')

In [ ]:
#Situações
manutencao = vendas.loc[(vendas['date_of_sales'] > pd.to_datetime('2019-06-01')) & (vendas['date_of_sales'] < pd.to_datetime('2019-07-01'))].groupby(by=['date_of_sales', 'storeid']).agg({'sales_qty': 'sum'}).reset_index().groupby(by=['storeid']).agg(vendas_manutencao=('sales_qty', 'mean'))
condicao_ultima_semana = vendas.loc[vendas['date_of_sales'] > pd.to_datetime('2019-07-01')].groupby(by=['date_of_sales','storeid']).sum().reset_index().groupby(by=['storeid']).agg(vendas_ultima_semana=('sales_qty', 'mean'))
condicoes_preexistentes=pd.merge(manutencao,condicao_ultima_semana,left_index=True,right_index=True,how='outer')
for col in ['vendas_manutencao', 'vendas_ultima_semana']:
  condicoes_preexistentes.loc[:, col] = condicoes_preexistentes.loc[:, col].fillna(0)

In [ ]:
# Visualização situacional
fig, ax = plt.subplots(nrows=3,sharex=True,figsize=(16, 15))
vendas_resumo = vendas.groupby(by=['date_of_sales', 'product']).agg(vendas=('sales_qty', 'sum')).reset_index()
vendas_resumo['medias'] = 0
estoque_resumo = pd.merge(estoque.groupby(by='year_month').agg({'supplied_qty': 'sum'}),vendas.groupby(by=['year_month']).agg({'sales_qty': 'sum'}),left_index=True,right_index=True,how='inner').reset_index()
estoque_resumo['data_ref'] = estoque_resumo.loc[:, 'year_month'].apply(lambda z: pd.to_datetime(z + '-15'))
estoque_resumo['saldo']=estoque_resumo['supplied_qty']-estoque_resumo['sales_qty']
for prod in vendas_resumo['product'].unique():
  vendas_resumo.loc[vendas_resumo['product']==prod,'medias']=vendas_resumo.loc[vendas_resumo['product']==prod,'vendas'].rolling(window=7, min_periods=1).mean()
sbn.lineplot(data=vendas_resumo, x='date_of_sales', y='vendas', hue='product', palette='Set2', ax=ax[0])
_=ax[0].set_ylabel("Vendas diárias por produto.")
sbn.lineplot(data=vendas_resumo, x='date_of_sales', y='medias', hue='product', palette='Set2', ax=ax[1])
_ = ax[1].set_ylabel("Média móvel de vendas de 7 dias")
sbn.lineplot(data=estoque_resumo, x='data_ref', y='supplied_qty', ax=ax[2], color='tab:cyan')
# ax[2].plot(estoque_resumo['data_ref'],estoque_resumo['supplied_data'],'')
axt = ax[2].twinx()
sbn.lineplot(data=estoque_resumo, x='data_ref', y='saldo', ax=axt, color='tab:red')
_ = ax[2].set_ylabel('Total reestocado')
_ = ax[2].set_xlabel('Data de refência')
_ = axt.set_ylabel('Saldo de Estoque')

In [ ]:
#Histograma graduado de variação
e_c = 0
e_m = 0.1
e_a = 0.5
fig,ax = plt.subplots(figsize=(16,10))
sbn.histplot(data=situacao_cidades, x='tendencia', binwidth=0.05, ax=ax)
ax.set_title('Crescimento do estoque nos municípios')
ax.set_ylabel("N° de cidades")
ax.set_xlabel("Intensidade do crescimento")
for pat in ax.patches:
  if pat.get_x() < e_c:
    pat.set_ec('darkred')
    pat.set_fc('tab:red')
  elif pat.get_x() < e_m:
    pat.set_ec('darkgreen')
    pat.set_fc('seagreen')
  elif pat.get_x() < e_a:
    pat.set_ec('darkslategrey')
    pat.set_fc('lightslategrey')
  else:
    pat.set_ec('darkorange')
    pat.set_fc('orange')

In [ ]:
for i in range(len(mapa_counties['features'])):
  fips = mapa_counties['features'][i]['properties']['FIPS_CODE']
  cidades_dentro_county = situacao_cidades[situacao_cidades['county_fips'] == fips]
  td = 'Sem vendas neste condado.'
  tend = '-'
  if len(cidades_dentro_county) > 0:
    tend = cidades_dentro_county['tendencia'].max() if cidades_dentro_county['tendencia'].min() >= 0 else cidades_dentro_county['tendencia'].min()
    td = "{:.2f} %".format(tend * 100)
  mapa_counties['features'][i]['properties']['Tendência'] = td
  mapa_counties['features'][i]['properties']['tend'] = tend
ob_mapa_counties = fl.Map(location=[39.833333, -98.585522], zoom_start=4)
fl.GeoJson(
  mapa_counties,
  name="Evolução do estoque dos condados",
  style_function=colorizar_county,
  tooltip=fl.GeoJsonTooltip(fields=('COUNTY_STATE_NAME','Tendência'),style="font-family: Roboto;")
).add_to(ob_mapa_counties)
ob_mapa_counties

In [ ]:
#Preencher o que der
preenchimento_das_vendas = pd.DataFrame()
todos_produtos = vendas['product'].unique()
#Maio
for mes in [5]:
  for dia in range(1, 32):
    data_ref = pd.to_datetime(f"2019-{mes:02n}-{dia:02n}")
    extrato = vendas.loc[vendas['date_of_sales'] == data_ref]
    if len(extrato) == 0:
      dia_da_semana = data_ref.dayofweek
      qto_dias_no_mes = [x for x in [pd.to_datetime(f"2019-{mes:02n}-{x:02n}") for x in range(1, 32)] if x.dayofweek == dia_da_semana]
      vendas_no_mes=pd.DataFrame()
      for loja in lojas['storeid'].unique():
        vendas_realizadas_no_mes = vendas[(vendas['storeid'] == loja) & (vendas['date_of_sales'].isin(qto_dias_no_mes))].copy()
        for prod in todos_produtos:
          if len(vendas_realizadas_no_mes[vendas_realizadas_no_mes['product'] == prod]) > 0:
            venda_media_dia_semana = vendas_realizadas_no_mes[vendas_realizadas_no_mes['product'] == prod]['sales_qty'].sum() / (len(qto_dias_no_mes)-1)
            vendas_no_mes = pd.concat([vendas_no_mes,
              pd.DataFrame.from_dict({0:[data_ref,prod,venda_media_dia_semana,loja]},columns=['date_of_sales','product','sales_qty','storeid'],orient='index')
            ],ignore_index=True)
      preenchimento_das_vendas = pd.concat([preenchimento_das_vendas, vendas_no_mes], ignore_index=True, copy=True, sort=True)
#Julho
for mes in [7]:
  for dia in range(20, 32):
    data_ref = pd.to_datetime(f"2019-{mes:02n}-{dia:02n}")
    dia_da_semana = data_ref.dayofweek
    datas_dos_dias_da_semana = [x for x in [pd.to_datetime(f"2019-{mes:02n}-{x:02n}") for x in range(1, 32)] if x.dayofweek == dia_da_semana]
    for loja in lojas['storeid'].unique():
      vendas_realizadas_nos_dias_da_semana = vendas[(vendas['date_of_sales'].dt.month == mes) & (vendas['date_of_sales'].isin(datas_dos_dias_da_semana)) & (vendas['storeid']==loja)]
      for prod in todos_produtos:
        if len(vendas_realizadas_nos_dias_da_semana[vendas_realizadas_nos_dias_da_semana['product'] == prod]) > 0:
          vendas_medias_observadas = vendas_realizadas_nos_dias_da_semana[vendas_realizadas_nos_dias_da_semana['product'] == prod].groupby(
            by=['storeid','product']
          ).agg('mean').reset_index()
          vendas_medias_observadas['date_of_sales']=data_ref
          preenchimento_das_vendas = pd.concat([preenchimento_das_vendas, vendas_medias_observadas], ignore_index=True)

In [ ]:
#Reconsolidar_tudo
vendas_compensadas = pd.concat([vendas.drop(labels=['year_month'], axis=1), preenchimento_das_vendas], ignore_index=True, copy=True, sort=True)
vendas_compensadas['year_month'] = vendas_compensadas['date_of_sales'].dt.strftime('%Y-%m')

In [ ]:
#Estoque estimado para agosto igual à reestocagem de julho
extrato_estoque_agosto = novo_estoque[novo_estoque['year_month'] == '2019-07'].copy()
extrato_estoque_agosto.loc[:,'year_month']='2019-08'
estoque_previsoes = pd.concat([novo_estoque, extrato_estoque_agosto], ignore_index=True, copy=True, sort=True)

In [ ]:
situacao_cidades_original = situar_cidades(vendas_compensadas, novo_estoque)

In [ ]:
# Se agosto for como maio
extrato_vendas_agosto = vendas_compensadas[vendas_compensadas['date_of_sales'].dt.month == 5].copy()
extrato_vendas_agosto.loc[:, 'date_of_sales'] = extrato_vendas_agosto.loc[:, 'date_of_sales'].apply(lambda z: pd.to_datetime(z.to_pydatetime().replace(month=8)))
extrato_vendas_agosto['year_month'] = '2019-08'
vendas_agosto_maio = pd.concat([vendas_compensadas, extrato_vendas_agosto], copy=True, sort=True, ignore_index=True)
situacao_cidades_agosto_maio = situar_cidades(vendas_agosto_maio,estoque_previsoes)

# Se agosto for como julho
extrato_vendas_agosto = vendas_compensadas[vendas_compensadas['date_of_sales'].dt.month == 7].copy()
extrato_vendas_agosto.loc[:, 'date_of_sales'] = extrato_vendas_agosto.loc[:, 'date_of_sales'].apply(lambda z: pd.to_datetime(z.to_pydatetime().replace(month=8)))
extrato_vendas_agosto['year_month'] = '2019-08'
vendas_agosto_julho = pd.concat([vendas_compensadas, extrato_vendas_agosto], copy=True, sort=True, ignore_index=True)
situacao_cidades_agosto_julho = situar_cidades(vendas_agosto_julho,estoque_previsoes)

# Se agosto for 50% a mais que julho
extrato_vendas_agosto.loc[:, 'sales_qty'] = extrato_vendas_agosto.loc[:, 'sales_qty'] * 1.5
vendas_agosto_julho_1_5 = pd.concat([vendas_compensadas, extrato_vendas_agosto], copy=True, sort=True, ignore_index=True)
situacao_cidades_agosto_julho_1_5 = situar_cidades(vendas_agosto_julho_1_5,estoque_previsoes)

In [ ]:
fig, ax = plt.subplots(nrows=4,ncols=1, figsize=(16, 20))
plotar_histograma_da_situacao_do_estoque_das_cidades(situacao_cidades_original, ax[0])
plotar_histograma_da_situacao_do_estoque_das_cidades(situacao_cidades_agosto_maio, ax[1])
plotar_histograma_da_situacao_do_estoque_das_cidades(situacao_cidades_agosto_julho, ax[2])
plotar_histograma_da_situacao_do_estoque_das_cidades(situacao_cidades_agosto_julho_1_5, ax[3])
for i, lab in enumerate([None, None, None, 'Tendência de variação do Estoque']):
  ax[i].set_xlabel(lab)
for i, lab in enumerate([
  'Estoques com preenchimento das vendas de julho',
  'Estoques caso as vendas em agosto estejam no patamar de maio',
  'Estoques caso as vendas em agosto estejam como o previsto para julho',
  'Estoques caso as vendas em agosto estejam 50% acima do previsto para julho'
]):
  ax[i].set_title(lab)

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=4, figsize=(16, 20), sharex=True)
plotar_vendas_diarias_e_estoque(vendas_compensadas,novo_estoque,ax[0])
plotar_vendas_diarias_e_estoque(vendas_agosto_maio, estoque_previsoes, ax[1])
plotar_vendas_diarias_e_estoque(vendas_agosto_julho, estoque_previsoes, ax[2])
plotar_vendas_diarias_e_estoque(vendas_agosto_julho_1_5, estoque_previsoes, ax[3])

In [ ]:
for i in range(len(mapa_counties['features'])):
  fips = mapa_counties['features'][i]['properties']['FIPS_CODE']
  cidades_dentro_county = situacao_cidades_original[situacao_cidades_original['county_fips'] == fips]
  td = 'Sem vendas neste condado.'
  tend = '-'
  if len(cidades_dentro_county) > 0:
    tend = cidades_dentro_county['tendencia'].max() if cidades_dentro_county['tendencia'].min() >= 0 else cidades_dentro_county['tendencia'].min()
    td = "{:.2f} %".format(tend * 100)
  mapa_counties['features'][i]['properties']['Tendência'] = td
  mapa_counties['features'][i]['properties']['tend'] = tend
ob_mapa_counties_original = fl.Map(location=[39.833333, -98.585522], zoom_start=4)
fl.GeoJson(
  mapa_counties,
  name="Evolução do estoque dos condados",
  style_function=colorizar_county,
  tooltip=fl.GeoJsonTooltip(fields=('COUNTY_STATE_NAME','Tendência'),style="font-family: Roboto;")
).add_to(ob_mapa_counties_original)
ob_mapa_counties_original

In [ ]:
for i in range(len(mapa_counties['features'])):
  fips = mapa_counties['features'][i]['properties']['FIPS_CODE']
  cidades_dentro_county = situacao_cidades_agosto_maio[situacao_cidades_agosto_maio['county_fips'] == fips]
  td = 'Sem vendas neste condado.'
  tend = '-'
  if len(cidades_dentro_county) > 0:
    tend = cidades_dentro_county['tendencia'].max() if cidades_dentro_county['tendencia'].min() >= 0 else cidades_dentro_county['tendencia'].min()
    td = "{:.2f} %".format(tend * 100)
  mapa_counties['features'][i]['properties']['Tendência'] = td
  mapa_counties['features'][i]['properties']['tend'] = tend
ob_mapa_counties_agosto_maio = fl.Map(location=[39.833333, -98.585522], zoom_start=4)
fl.GeoJson(
  mapa_counties,
  name="Evolução do estoque dos condados",
  style_function=colorizar_county,
  tooltip=fl.GeoJsonTooltip(fields=('COUNTY_STATE_NAME','Tendência'),style="font-family: Roboto;")
).add_to(ob_mapa_counties_agosto_maio)
ob_mapa_counties_agosto_maio

In [ ]:
for i in range(len(mapa_counties['features'])):
  fips = mapa_counties['features'][i]['properties']['FIPS_CODE']
  cidades_dentro_county = situacao_cidades_agosto_julho[situacao_cidades_agosto_julho['county_fips'] == fips]
  td = 'Sem vendas neste condado.'
  tend = '-'
  if len(cidades_dentro_county) > 0:
    tend = cidades_dentro_county['tendencia'].max() if cidades_dentro_county['tendencia'].min() >= 0 else cidades_dentro_county['tendencia'].min()
    td = "{:.2f} %".format(tend * 100)
  mapa_counties['features'][i]['properties']['Tendência'] = td
  mapa_counties['features'][i]['properties']['tend'] = tend
ob_mapa_counties_agosto_julho = fl.Map(location=[39.833333, -98.585522], zoom_start=4)
fl.GeoJson(
  mapa_counties,
  name="Evolução do estoque dos condados",
  style_function=colorizar_county,
  tooltip=fl.GeoJsonTooltip(fields=('COUNTY_STATE_NAME','Tendência'),style="font-family: Roboto;")
).add_to(ob_mapa_counties_agosto_julho)
ob_mapa_counties_agosto_julho

In [ ]:
for i in range(len(mapa_counties['features'])):
  fips = mapa_counties['features'][i]['properties']['FIPS_CODE']
  cidades_dentro_county = situacao_cidades_agosto_julho_1_5[situacao_cidades_agosto_julho_1_5['county_fips'] == fips]
  td = 'Sem vendas neste condado.'
  tend = '-'
  if len(cidades_dentro_county) > 0:
    tend = cidades_dentro_county['tendencia'].max() if cidades_dentro_county['tendencia'].min() >= 0 else cidades_dentro_county['tendencia'].min()
    td = "{:.2f} %".format(tend * 100)
  mapa_counties['features'][i]['properties']['Tendência'] = td
  mapa_counties['features'][i]['properties']['tend'] = tend
ob_mapa_counties_agosto_julho_1_5 = fl.Map(location=[39.833333, -98.585522], zoom_start=4)
fl.GeoJson(
  mapa_counties,
  name="Evolução do estoque dos condados",
  style_function=colorizar_county,
  tooltip=fl.GeoJsonTooltip(fields=('COUNTY_STATE_NAME','Tendência'),style="font-family: Roboto;")
).add_to(ob_mapa_counties_agosto_julho_1_5)
ob_mapa_counties_agosto_julho_1_5